In [3]:
#import jax.numpy as np
import matplotlib.pyplot as plt
#from scipy.optimize import minimize
import numpy as np
#plt.rcParams["figure.figsize"] = (25,25)
from jax.config import config
config.update("jax_enable_x64", True)
import networkx as nx
from scipy.optimize import minimize
import jax.numpy as nnp
from jax import grad
import random

import laplacians as lp
import opt_helpers as oh


data

In [4]:
#numbers of edges connecting part and graph
n_con=0

#full graph including the part
full_graph='data/can_processed_'+str(n_con)+'.txt'
#part of the graph
part_graph='data/can_sub_'+str(n_con)+'.txt'
#nodes of the part of the graph
nodes_part='data/can_subnodes_0.txt'

initialization - ground truth plus uniformly distributed noise. Rest of the graph gets initialized with $rest$, with noise in intervall $[-\frac{noise}{2},\frac{noise}{2}]$ added.


In [5]:
diag_val=0.0
noise=5.0
rest=1.0

#damping factor
mu=1.0

A=lp.edgelist_to_adjmatrix(full_graph)
full_size=np.shape(A)[0]


subset_nodes=np.loadtxt(nodes_part).astype(int)-1
x0=lp.init_orig_subgraph(subset_nodes,full_size, noise,rest)

diag_init=np.diag(x0)
x0=np.ndarray.flatten(x0)

load/prepare real graph + make colormap for rest and part

In [6]:
A = lp.edgelist_to_adjmatrix(full_graph)
A_part= lp.edgelist_to_adjmatrix(part_graph)


G=nx.from_numpy_matrix(A)


G_part=nx.from_numpy_matrix(A_part)


well=nx.to_numpy_matrix(G)


#colormap

subset_nodes=np.loadtxt(nodes_part).astype(int)-1
G_part=nx.from_numpy_matrix(A_part)
color_map=[]
for node in G:
    if node in subset_nodes:
        color_map.append('blue')
    else: 
        color_map.append('green')     
        

calculate Laplacians and Eigendecomposition

In [7]:


size_part=G_part.number_of_nodes()
size_rest=G.number_of_nodes()-size_part

full_size=size_part + size_rest

A_full = nx.adjacency_matrix(G)
A_part = nx.adjacency_matrix(G_part)

Deg_part= lp.make_degree_matrix(A_part)
Deg_full = lp.make_degree_matrix(A_full)

    #calculate Laplacians
L_full, D_full, V_full= lp.decompose_laplacian_unnorm(A_full, Deg_full, size_part+size_rest)
L_part, D_part, V_part= lp.decompose_laplacian_unnorm(A_part, Deg_part,size_part)

    
   

proximal operator of l21- norm

In [33]:
def prox_l21(X,lamb):
    res=0.0
    for i in range(np.shape(X)[0]):
        res=res+prox_l2(X[i,:],lamb)
    return res

def prox_l2(x,lamb):
    return np.maximum(1.0-(lamb/np.linalg.norm(x, ord=2.0)),0.0)*x

test

In [34]:
X=np.random.rand(10,10)
print(X)
print("--------------")
print(prox_l21(X,0.1))

[[0.61995605 0.19698273 0.07149614 0.83245972 0.80862241 0.14531751
  0.07224188 0.10831237 0.15553789 0.78823915]
 [0.18305134 0.976946   0.94075472 0.84546961 0.52915441 0.06189722
  0.06643525 0.58982571 0.62500367 0.097574  ]
 [0.57143293 0.03312445 0.67777551 0.48234072 0.85420176 0.467167
  0.12700606 0.05149184 0.22991501 0.27651606]
 [0.20067943 0.5660133  0.97139104 0.66751767 0.83236486 0.4432028
  0.64507261 0.31810507 0.78163462 0.71801551]
 [0.64241075 0.65286283 0.2200348  0.13031271 0.6311381  0.24264858
  0.41085299 0.67970607 0.1882885  0.25735901]
 [0.1252655  0.91194565 0.93711558 0.07287486 0.90960044 0.4685104
  0.97484401 0.34553922 0.40273666 0.94347007]
 [0.19811369 0.63288947 0.43636825 0.24460249 0.9985825  0.45137264
  0.0478147  0.57720886 0.73851048 0.87727788]
 [0.76099241 0.26802905 0.40965378 0.77629629 0.53604216 0.07752549
  0.77130867 0.95613854 0.18012436 0.48792742]
 [0.23390787 0.43515092 0.40254654 0.2872278  0.35694805 0.08079882
  0.78850823 0.9